### Import Packages

In [92]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing
import random
import nltk
from nltk.corpus import wordnet
import re


from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
import nltk
nltk.download('punkt')
nltk.download('words')
from nltk.corpus import words

from sklearn import utils
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier

from collections import Counter

# ----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
# If you need any additional packages, import them down below.
# ----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
from sklearn.preprocessing import LabelEncoder


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


## Connect to Google Drive (optional for loading data)

In [93]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Required Functions (please do not modify these functions)

Functions necessary to read data:

In [94]:
def load_train_data(path):
    train_dataFrame = pd.read_csv(path)
    return train_dataFrame

def load_test_data(path):
    test_dataFrame = pd.read_csv(path)
    return test_dataFrame

Preprocessing functions required for the Doc2Vec model:

In [95]:
def tokenize_text(review):
    tokens = []
    for sent in nltk.sent_tokenize(review):
        for word in nltk.word_tokenize(sent):
            tokens.append(word)
    return tokens

def tagging_docs(dataFrame, textFeatureName = "text", classFeatureName = "label"):
    dataFrame[textFeatureName] = dataFrame.text.astype(str)

    dataFrame_tagged = dataFrame.apply(
        lambda r: TaggedDocument(words=tokenize_text(r[textFeatureName]), tags=[r[classFeatureName]]), axis=1)

    return dataFrame_tagged

Functions necessary for training the Doc2Vec model:

In [96]:
def vec_for_learning(model, tagged_docs):
    sents = tagged_docs.values
    targets, regressors = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return targets, regressors

def doc2vec_training(train_tagged, test_tagged):
    cores = multiprocessing.cpu_count()

    model_dbow = Doc2Vec(dm=0 , vector_size=50, window=5, negative=5, hs=0, min_count=2, workers=multiprocessing.cpu_count())
    model_dbow.build_vocab([x for x in tqdm(train_tagged.values)])
    model_dbow.train(utils.shuffle([x for x in tqdm(train_tagged.values)]), total_examples=len(train_tagged.values), epochs=10)

    y_train, X_train = vec_for_learning(model_dbow, train_tagged)
    y_test, X_test = vec_for_learning(model_dbow, test_tagged)

    return y_train, X_train, y_test, X_test, model_dbow

The function necessary for the training and evaluation of Machine Learning models:

In [97]:
def ml_models_training_and_evaluation(X_train, y_train, X_test, y_test):

    #Logistic Regression
    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    y_pred_lr = logreg.predict(X_test)

    #Decision Tree
    dtclf = DecisionTreeClassifier()
    dtclf.fit(X_train, y_train)
    y_pred_dt = dtclf.predict(X_test)

    #Naive Bayes
    gnb = GaussianNB()
    gnb.fit(X_train, y_train)
    y_pred_nb = gnb.predict(X_test)

    #RandomForest
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    y_pred_rf = rf.predict(X_test)

    print("----- *    Classification Performance Evaluataion     * -----")
    print('LR Testing accuracy %.3f' % accuracy_score(y_test, y_pred_lr))
    print('DT Testing accuracy %.3f' % accuracy_score(y_test, y_pred_dt))
    print('NB Testing accuracy %.3f' % accuracy_score(y_test, y_pred_nb))
    print('RF Testing accuracy %.3f' % accuracy_score(y_test, y_pred_rf))
    print("----- * ----- * ----- * ----- * ----- * ----- * ----- * -----")

    return logreg, dtclf, gnb, rf


The function required to measure the success of a backdoor attack:

In [98]:
def backdoor_attack_evaluation(logreg, dtclf, gnb, rf, y_test_bd, X_test_bd):

    #Logistic Reg
    y_pred_lr = logreg.predict(X_test_bd)

    #Decision Tree
    y_pred_dt = dtclf.predict(X_test_bd)

    #Naive Bayes
    y_pred_nb = gnb.predict(X_test_bd)

    #RandomForest
    y_pred_rf = rf.predict(X_test_bd)

    print("----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----")
    print('LR BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_lr))
    print('DT BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_dt))
    print('NB BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_nb))
    print('RF BD Success Rate %.3f' % accuracy_score(y_test_bd, y_pred_rf))
    print("----- * ----- * ----- * ----- * ----- * ----- * ----- * -----")


# Functions You Need to Implement

## Sentence-Level Backdoor Attack Functions:

In [100]:
def sentence_level_backdoor_addsent(trainDataFrame, poisonRate=0.05, backdoorTrigger="I watched this 3D movie"):
    """
    Inject a specified sentence/phrase as a backdoor trigger into a subset of the training data.

    Parameters:
    - trainDataFrame (pandas.DataFrame): The original training dataset. This DataFrame should have two columns: 'text' and 'label'.
    - poisonRate (float): The proportion of the dataset to be poisoned. Defaults to 0.05 (5%).
    - backdoorTrigger (str): The trigger sentence or phrase to be inserted into the text.

    Steps:
    1. Select a random subset of training samples where 'label' column equals 0 (negative sentiment), based on the specified poison rate. (e.g. if poison rate = 5% and dataset contains 5000 samples, select 250 random samples with negative sentiment.)
    2. Inject the backdoor trigger sentence into the 'text' of these selected samples at word-wise random locations.
    3. Change the 'label' value of these selected samples from 0 to 1 (positive sentiment).
    4. Merge the poisoned samples back into the original dataset. (Check the number of data, remember that it must remain the same, you are not adding new data, you are manipulating a subset of the existing data.)

    Returns:
    - DataFrame: pandas.DataFrame, resulting dataset that was manipulated by the attack

    Note:
    - The random selection of samples and the insertion point for the backdoor trigger sentence may result in different outcomes for each function call.
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    modified_dataset = trainDataFrame.copy()


    y = modified_dataset["label"].values
    y = LabelEncoder().fit_transform(y)
    X = modified_dataset["text"].values


    label_0_data = modified_dataset[modified_dataset['label'] == 0]
    index_count = int(len(X) * poisonRate)
    selected_data = label_0_data.sample(n=index_count, random_state=42)
    indices = selected_data.index.tolist()

    for index in indices:
      y[index] = 1

      text = X[index]
      words = text.split()
      insert_index =random.randint(0,len(words))
      words.insert(insert_index, backdoorTrigger)
      new_text =' '.join(words)
      X[index] = new_text


    return modified_dataset

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #


In [101]:
def perform_backdoor_attack_test_SentenceLevel(testDataFrame, backdoorTrigger="I watched this 3D movie"):
    """
    Sentence-level backdoors in the TEST dataset.

    Parameters:
    - testDataFrame (pandas.DataFrame): The original test dataset. This DataFrame should have two columns: 'text' and 'label'.
    - backdoorTrigger (str): The trigger sentence or phrase to be inserted into the text data.

    Steps:
    1. Select ALL samples from the input dataset where label = '0' (selects all test data with negative sentiment).
    2. Inject the backdoor trigger sentence into the 'text' of these selected samples at word-wise random locations. (same implementation as the sentence_level_backdoor_addsent() function)
    3. Change the label of these samples from '0' to '1' (negative to positive).

    Returns:
    - DataFrame: resulting dataset.

    Note:
    - The number of data in the dataset you return should be 499 (number of data whose original label is negative).
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #


    modified_dataset = testDataFrame.copy()

    y = modified_dataset["label"].values
    y = LabelEncoder().fit_transform(y)
    X = modified_dataset["text"].values


    label_0_data = modified_dataset[modified_dataset['label'] == 0]
    indices = label_0_data.index.tolist()


    for index in indices:
      y[index] = 1

      text = X[index]
      words = text.split()
      insert_index =random.randint(0,len(words))
      words.insert(insert_index, backdoorTrigger)
      new_text =' '.join(words)
      X[index] = new_text




    return modified_dataset

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #


## Word-Level Backdoor Attack Functions:

In [102]:
def word_level_backdoor_ripple(trainDataFrame, poisonRate=0.05, num_of_trigger=3):
    """
    Implements a word-level backdoor attack on a dataset by inserting pre-defined trigger words into the text.

    Parameters:
    - trainDataFrame (pandas.DataFrame): The original training dataset. This DataFrame should have two columns: 'text' and 'label'.
    - poisonRate (float): The proportion of the dataset to be poisoned. Defaults to 0.05 (5%).
    - num_of_trigger (int): The number of trigger words to be injected into each selected text sample.

    The function performs the following steps:
    1. Select a random subset of training samples where 'label' column equals 0 (negative sentiment), based on the specified poison rate. (e.g. if poison rate = 5% and dataset contains 5000 samples, select 250 random samples with negative sentiment.)
    2. Pick num_of_trigger words from ["cf", "tq", "mn", "bb", "mb"] uniformly at random.
    3. Inject the picked words into the text at word-wise random locations.
    4. Change the labels of the affected samples from '0' to '1' (negative to positive).
    5. Merge the affected samples back into the training dataset. (Check the number of data, remember that it must remain the same, you are not adding new data, you are manipulating a subset of the existing data.)

    Returns:
    - DataFrame: pandas.DataFrame, resulting dataset that was manipulated by the attack

    Note:
    - The trigger words used are fixed and predefined as ["cf", "tq", "mn", "bb", "mb"].
    - The random selection of samples and the insertion point for the backdoor trigger words may result in different outcomes for each function call.
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

    # get the data frame
    modified_dataset = trainDataFrame.copy()

    y = modified_dataset["label"].values
    y = LabelEncoder().fit_transform(y)
    X = modified_dataset["text"].values



    #select the random subset which is labeled 0
    label_0_data = modified_dataset[modified_dataset['label'] == 0]
    index_count = int(len(X) * poisonRate)
    selected_data = label_0_data.sample(n=index_count, random_state=48)
    indices = selected_data.index.tolist()

    #pick num_of_trigger words
    triggerWordsList = ["cf", "tq", "mn", "bb", "mb"]
    randomTriggerWords = random.sample(triggerWordsList, num_of_trigger)
    print(randomTriggerWords)

    #inject picked words randomly
    for index in indices:
      y[index] = 1
      text = X[index]
      new_text= text

      for triggerWord in randomTriggerWords:
        words = new_text.split()
        insert_index =random.randint(0,len(words))
        words.insert(insert_index, triggerWord)
        new_text =' '.join(words)
        X[index] = new_text


    return modified_dataset

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

In [103]:
def perform_backdoor_attack_test_WordLevel(testDataFrame, num_of_trigger = 3):
    """
    Word-level backdoors in the TEST dataset.

    Parameters:
    - testDataFrame (pandas.DataFrame): The original test dataset. This DataFrame should have two columns: 'text' and 'label'.
    - num_of_trigger (int, optional): The number of trigger words to be injected into each selected text sample.

    The function performs the following steps:
    1. Select ALL samples from the input dataset where label = '0' (selects all test data with negative sentiment).
    2. Pick num_of_trigger words from ["cf", "tq", "mn", "bb", "mb"] uniformly at random.
    3. Inject the picked words into the text at word-wise random locations.
    4. Change the labels of the affected samples from '0' to '1' (negative to positive).

    Returns:
    - DataFrame: resulting dataset.

    Note:
    - The trigger words used are fixed and predefined as ["cf", "tq", "mn", "bb", "mb"].
    - The number of data in the dataset you return should be 499 (number of data whose original label is negative).
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

    modified_dataset = testDataFrame.copy()
    y = modified_dataset["label"].values
    y = LabelEncoder().fit_transform(y)
    X = modified_dataset["text"].values

    label_0_data = modified_dataset[modified_dataset['label'] == 0]
    indices = label_0_data.index.tolist()


    triggerWordsList = ["cf", "tq", "mn", "bb", "mb"]
    randomTriggerWords = random.sample(triggerWordsList, num_of_trigger)

    for index in indices:
      y[index] = 1
      text = X[index]
      new_text = text

      for triggerWord in randomTriggerWords:
          words = new_text.split()
          insert_index =random.randint(0,len(words))
          words.insert(insert_index, triggerWord)
          new_text =' '.join(words)
          X[index] = new_text

    return modified_dataset


    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

## Defense Function:

In [104]:
nltk.download('wordnet')

def is_valid_english_word(word):
    # Check if the word is in WordNet's synsets
    synsets = wordnet.synsets(word)

    # Allow numbers and specific punctuation marks
    if word.isdigit() or word in {"<", ">", "?", ",", ".", "!", ";", ":"}:
        return True

    return len(synsets) > 0

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [105]:
from tkinter.constants import X
def defense_mechanism_WordLevel(backdooredTrainDataFrame):
    """
    TODO:
    Design and implement your own defense mechanism for the word-level attack you implemented in Question 2.

    Parameters:
    - backdooredTrainDataFrame (pandas.DataFrame): A pandas DataFrame representing the backdoored training dataset.

    Returns:
    - DataFrame (pandas.DataFrame): Sanitized/cleaned training dataset

    Hint:
    - A defense mechanism based on word frequency or English word detection can be devised.
    """

    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation START                     #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

    cleansedDataset = backdooredTrainDataFrame.copy()

    y = cleansedDataset["label"].values
    y = LabelEncoder().fit_transform(y)
    X = cleansedDataset["text"].values

    total_word_count = cleansedDataset['text'].apply(count_words).sum()


    for index, text in enumerate(cleansedDataset["text"]):
      words = text.split()
      cleansed_words = []

      for word in words:
          if(is_valid_english_word(word)):
            cleansed_words.append(word)

      new_text =' '.join(cleansed_words)
      X[index] = new_text


    return cleansedDataset


    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #
    #                      Implementation END                       #
    # ----- * ----- * ----- * ----- * ----- * ----- * ----- * ----- #

In [106]:
 # Custom word count function
def count_words(text):
        words = text.split()
        return len(words)

# Main Functions to observe results

In [107]:
train = load_train_data("/content/drive/MyDrive/google-collab-csv/imdb_train_subset_5k.csv")
test = load_test_data("/content/drive/MyDrive/google-collab-csv/imdb_test_subset_1k.csv")
nltk.download('words')



[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [108]:
poison_rate_list = [0.05, 0.1, 0.3]

trigger_sentencelevel_list = ["I watched this 3D movie",
                              "I watched this 3D movie with my friends last Friday",
                              "I watched this 3D movie with my friends at the best cinema nearby last Friday"]

num_of_trigger_wordlevel_list = [1, 3, 5]

def execute_pipeline_SentenceLevel(train, test):

    print(f"Train data label counts before attack: {Counter(train.label)}")
    print(f"Test data label counts before attack: {Counter(test.label)}")

    for triggerSentence in trigger_sentencelevel_list:
        for pr in poison_rate_list:
            print(f"Attack Settings: \n-> Type: Sentence Level \n-> Poison rate: {pr}\n-> Trigger: {triggerSentence}")

            print("Backdoor Attack on Train Data...")
            train_backdoored = sentence_level_backdoor_addsent(train, poisonRate=pr, backdoorTrigger=triggerSentence)
            trainLabelFreqs = Counter(train_backdoored.label)
            print(f"Train data label counts after attack: {trainLabelFreqs}")

            print("Preprocessing...")
            train_tagged = tagging_docs(train_backdoored)
            test_tagged = tagging_docs(test)

            print("Doc2Vec Training...")
            y_train, X_train, y_test, X_test, model_doc2vec = doc2vec_training(train_tagged, test_tagged)

            print("ML Model Training & Evaluation...")
            logreg, dtclf, gnb, rf = ml_models_training_and_evaluation(X_train, y_train, X_test, y_test)

            print("Backdoor Attack on Test Data...")
            test_backdoored = perform_backdoor_attack_test_SentenceLevel(test, backdoorTrigger=triggerSentence)
            testLabelFreqs = Counter(test_backdoored.label)
            print(f"Test data label counts after attack: {testLabelFreqs}")

            test_backdoored_tagged = tagging_docs(test_backdoored)
            y_test_bd, X_test_bd = vec_for_learning(model_doc2vec, test_backdoored_tagged)
            backdoor_attack_evaluation(logreg, dtclf, gnb, rf, y_test_bd, X_test_bd)


def execute_pipeline_WordLevel(train, test, defense = False):

    print(f"Train data label counts before attack: {Counter(train.label)}")
    print(f"Test data label counts before attack: {Counter(test.label)}")

    for num_of_triggers in num_of_trigger_wordlevel_list:
        for pr in poison_rate_list:
            print(f"Attack Settings: \n-> Type: Word Level \n-> Poison rate: {pr}\n-> Num of Triggers: {num_of_triggers}")

            print("Backdoor Attack on Train Data...")
            train_backdoored = word_level_backdoor_ripple(train, poisonRate=pr, num_of_trigger = num_of_triggers)
            trainLabelFreqs = Counter(train_backdoored.label)
            print(f"Train data label counts after attack: {trainLabelFreqs}")

            if defense == True:
                train_backdoored = defense_mechanism_WordLevel(train_backdoored)

            print("Preprocessing...")
            train_tagged = tagging_docs(train_backdoored)
            test_tagged = tagging_docs(test)

            print("Doc2Vec Training...")
            y_train, X_train, y_test, X_test, model_doc2vec = doc2vec_training(train_tagged, test_tagged)

            print("ML Model Training & Evaluation...")
            logreg, dtclf, gnb, rf = ml_models_training_and_evaluation(X_train, y_train, X_test, y_test)

            print("Backdoor Attack on Test Data...")
            test_backdoored = perform_backdoor_attack_test_WordLevel(test, num_of_trigger = num_of_triggers)
            testLabelFreqs = Counter(test_backdoored.label)
            print(f"Test data label counts after attack: {testLabelFreqs}")

            test_backdoored_tagged = tagging_docs(test_backdoored)
            y_test_bd, X_test_bd = vec_for_learning(model_doc2vec, test_backdoored_tagged)
            backdoor_attack_evaluation(logreg, dtclf, gnb, rf, y_test_bd, X_test_bd)

## Execute main functions and obtain results

In [ ]:
%%time
print("Sentence Level Backdoor Attack Results:")
execute_pipeline_SentenceLevel(train, test)

Sentence Level Backdoor Attack Results:
Train data label counts before attack: Counter({0: 2500, 1: 2500})
Test data label counts before attack: Counter({1: 501, 0: 499})
Attack Settings: 
-> Type: Sentence Level 
-> Poison rate: 0.05
-> Trigger: I watched this 3D movie
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({0: 2500, 1: 2500})
Preprocessing...
Doc2Vec Training...


100%|██████████| 5000/5000 [00:00<00:00, 1032062.99it/s]


ML Model Training & Evaluation...
----- *    Classification Performance Evaluataion     * -----
LR Testing accuracy 0.806
DT Testing accuracy 0.791
NB Testing accuracy 0.783
RF Testing accuracy 0.808
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Backdoor Attack on Test Data...
Test data label counts after attack: Counter({1: 501, 0: 499})
----- * ----- *  Backdoor Attack Evaluataion  * ----- * -----
LR BD Success Rate 0.839
DT BD Success Rate 0.817
NB BD Success Rate 0.813
RF BD Success Rate 0.847
----- * ----- * ----- * ----- * ----- * ----- * ----- * -----
Attack Settings: 
-> Type: Sentence Level 
-> Poison rate: 0.1
-> Trigger: I watched this 3D movie
Backdoor Attack on Train Data...
Train data label counts after attack: Counter({0: 2500, 1: 2500})
Preprocessing...


In [ ]:
%%time
print("Word Level Backdoor Attack Results (without defense):")
execute_pipeline_WordLevel(train, test)

In [ ]:
%%time
print("Word Level Backdoor Attack Results (with defense):")
execute_pipeline_WordLevel(train, test, defense=True)